imoportando bibliotecas

In [2]:
import pandas as pd
import openpyxl
import awswrangler as awr

gerando dfs

In [ ]:
read_path = r"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\CAMPANHA_RANKING_ATIVACOES.xlsx"

df_ativ = pd.read_excel(read_path, engine='openpyxl', sheet_name='ATIVAÇÕES')
df_cancel = pd.read_excel(read_path, engine='openpyxl', sheet_name='CANCELAMENTOS')



,placa,chassi,id_placa,id_veiculo,id_carroceria,matricula,conjunto,unidade,status,cliente,data_registro,data_cancelamento,suporte,data_extracao,empresa
0,LUG4B30,93ZA01BDZL8936040,184961,184961,0,2046,2638,TS CONSULTORIA EM TRANSPORTES LTDA,CANCELADO,MV EDUARDA EMPREENDIMENTOS LTDA,2024-08-29,2025-01-02,Débora Mello,2025-03-11,Viavante
1,RJS0A93,9BVRG20C5LE874910,148919,148919,0,43766,136562,UNIDADE FEIRA DE SANTANA,CANCELADO,TRANSMORENO TRANSPORTADORA E COMERCIO DE BEBID...,2024-01-09,2025-01-02,Débora Mello,2025-03-11,Segtruck
2,BDD0H14,9BSR6X400K3954025,185001,185001,0,2053,2653,UNIDADE RONDONÓPOLIS,CANCELADO,BACCI PRESTACAO DE SERVICOS LTDA,2024-08-29,2025-01-02,Débora Mello,2025-03-11,Viavante
3,HRS1043,9A90708203SAC6265,8931,0,8931,16034,3881,UNIDADE ITAPEJARA,FINALIZADO,MATEUS TRANSPORTES LTDA,2023-12-14,2025-01-02,System,2025-03-11,Stcoop
4,GFV4A90,9BVRG20C5GE840794,13961,13961,0,16034,3881,UNIDADE ITAPEJARA,FINALIZADO,MATEUS TRANSPORTES LTDA,2023-12-14,2025-01-02,System,2025-03-11,Stcoop


criando dfs filtrados

In [28]:
#setando para datetime com pandas
df_cancel['data_cancelamento']=pd.to_datetime(df_cancel['data_cancelamento'])

df_ativ['data_ativacao'] = pd.to_datetime(df_ativ['data_ativacao'])


#filtrando datas
df_cancel_1trim = df_cancel[df_cancel['data_cancelamento'].between('2025-01-01', '2025-03-31')]

df_ativ_2trim = df_ativ[df_ativ['data_ativacao'].dt.month == 4]

#filtrando empresas

df_cancel_1trim_seg_st = df_cancel_1trim[df_cancel_1trim['empresa'].isin(['Segtruck','Stcoop'])]

df_ativ_2trim_via = df_ativ_2trim[df_ativ_2trim['empresa']=='Viavante']


#simplificando nomes
df_cancel = df_cancel_1trim_seg_st

df_ativ = df_ativ_2trim_via




conectando com sqlite3

In [29]:
import sqlite3

conn  = sqlite3.connect(":memory:") #criando banco temporário

df_cancel.to_sql("canceladas", conn, index=False, if_exists='replace')

df_ativ.to_sql("ativadas", conn, index=False, if_exists='replace')




2848

query

In [ ]:
query = """
SELECT 
    c.placa,
    c.chassi,
    c.id_placa,
    c.id_veiculo,
    c.id_carroceria,
    c.matricula,
    c.conjunto,
    c.unidade,
    c.data_cancelamento,
    a.data_ativacao
FROM canceladas AS c
INNER JOIN ativadas AS a ON a.placa = c.placa
"""

df_cancel_ativ = pd.read_sql_query(query,conn)

df_cancel_ativ['data_ativacao'] = pd.to_datetime(df_cancel_ativ['data_ativacao'])
df_cancel_ativ['data_ativacao'] = df_cancel_ativ['data_ativacao'].dt.date

df_cancel_ativ['data_cancelamento'] = pd.to_datetime(df_cancel_ativ['data_cancelamento'])
df_cancel_ativ['data_cancelamento'] = df_cancel_ativ['data_cancelamento'].dt.date

df_cancel_ativ.head()

,placa,chassi,id_placa,id_veiculo,id_carroceria,matricula,conjunto,unidade,data_cancelamento,data_ativacao
0,CPN2J91,9BVAS50D8BE765732,49334,49334,0,12001,136549,UNIDADE LONDRINA,2025-01-07,2025-04-04
1,NWI3692,9A9BT3112BCDJ5317,18856,0,18856,22240,12643,UNIDADE CASCAVEL,2025-01-10,2025-04-22
2,OMR5D06,9BVAG20C3CE793762,27737,27737,0,22240,12643,UNIDADE CASCAVEL,2025-01-10,2025-04-22
3,ATA9D05,9BSG4X200A3669408,3259,3259,0,21097,12693,UNIDADE MARINGÁ,2025-01-17,2025-04-02
4,QCF1A14,94BL0262KKR004426,11203,0,11203,20191,5096,UNIDADE SINOP,2025-01-23,2025-04-17


verificação de meses

In [33]:
df_cancel_ativ['data_cancelamento'] = pd.to_datetime(df_cancel_ativ['data_cancelamento'])
df_cancel_ativ['data_ativacao'] = pd.to_datetime(df_cancel_ativ['data_ativacao'])

meses_cancel = df_cancel_ativ['data_cancelamento'].dt.month.unique()
meses_ativ = df_cancel_ativ['data_ativacao'].dt.month.unique()

print(meses_cancel)
print(meses_ativ)

[1 2 3]
[4]
